# Floating potential approach

The currents are imposed and the floating potential is determined.
In addition, one electrode is set to ground.

An efficient block preconditioner needs to be found to solve this kind of system
with an iterative solver.

In [ ]:
import netgen.occ as occ
from netgen.webgui import Draw as DrawGeo
import ngsolve
from ngsolve.webgui import Draw

In [ ]:
unit_square = occ.unit_square_shape

In [ ]:
e1 = occ.Circle((0.2, 0.2), r=0.1)
e2 = occ.Circle((0.5, 0.5), r=0.1)
e3 = occ.Circle((0.8, 0.8), r=0.1)

In [ ]:
electrodes = [e1.Face(), e2.Face(), e3.Face()]
for idx, electrode in enumerate(electrodes):
    for edge in electrode.edges:
        edge.name = "Electrode_{}".format(idx)
    unit_square = unit_square - electrode

In [ ]:
DrawGeo(unit_square)

In [ ]:
mesh = ngsolve.Mesh(occ.OCCGeometry(unit_square, dim=2).GenerateMesh())
mesh.Curve(2)

In [ ]:
Draw(mesh)
print(mesh.GetBoundaries())

In [ ]:
V = ngsolve.H1(mesh, order=2, dirichlet="Electrode_2")

boundaries = ["Electrode_{}".format(idx) for idx in range(2)]
plateaus = []
for boundary in boundaries:
    plateaus.append(mesh.Boundaries(boundary))

fes = ngsolve.PlateauFESpace(V, plateaus)

boundaries.append("Electrode_2")

I_1 = -0.5  # A
I_2 = -0.5
I_3 = 1.0
I = [I_1, I_2, I_3]

u = fes.TrialFunction()
v = fes.TestFunction()

a = ngsolve.BilinearForm(fes)
a += ngsolve.grad(u) * ngsolve.grad(v) * ngsolve.dx

f = ngsolve.LinearForm(fes)
f += 0.0 * v * ngsolve.dx
for I_x, boundary in zip(I, boundaries):
    length = ngsolve.Integrate(ngsolve.CoefficientFunction(1.0) * ngsolve.ds(boundary), mesh)
    f += I_x / length * v * ngsolve.ds(boundary)

a.Assemble()
f.Assemble()

gfu = ngsolve.GridFunction(fes)

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
Draw(gfu)

# Check output

Try to understand orientation of normal vector!

In [ ]:
normal_component = ngsolve.specialcf.normal(2)
current_density = ngsolve.BoundaryFromVolumeCF(-ngsolve.grad(gfu)) * normal_component
print(ngsolve.Integrate(current_density * ngsolve.ds("Electrode_0"), mesh))
print(ngsolve.Integrate(current_density * ngsolve.ds("Electrode_1"), mesh))
print(ngsolve.Integrate(current_density * ngsolve.ds("Electrode_2"), mesh))

# Floating potentials

In [ ]:
print(ngsolve.Integrate(gfu * ngsolve.ds("Electrode_0"), mesh) / ngsolve.Integrate(ngsolve.CoefficientFunction(1.0) * ngsolve.ds("Electrode_0"), mesh))
print(ngsolve.Integrate(gfu * ngsolve.ds("Electrode_1"), mesh) / ngsolve.Integrate(ngsolve.CoefficientFunction(1.0) * ngsolve.ds("Electrode_1"), mesh))
print(ngsolve.Integrate(gfu * ngsolve.ds("Electrode_2"), mesh) / ngsolve.Integrate(ngsolve.CoefficientFunction(1.0) * ngsolve.ds("Electrode_2"), mesh))

# Field plot

In [ ]:
Draw(-ngsolve.grad(gfu), mesh)